In [1]:
import pandas as pd
import os
import numpy as np
import pickle
import json
import scipy
import matplotlib.pyplot as plt
import shutil
from skimage.measure import regionprops
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
import sys
import string
from PIL import Image
from scipy import signal
from scipy.interpolate import interp1d
from skimage import measure
import seaborn as sns
from scipy import stats
from skimage.draw import rectangle
import importlib
import splitfolders
from scipy.stats import gaussian_kde
import sklearn
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


import plots
import df_utils
importlib.reload(df_utils)
import features
import Figure2plots
importlib.reload(Figure2plots)

<module 'Figure2plots' from 'C:\\Users\\codyt\\Documents\\repos\\time-series-dc\\Figure2plots.py'>

In [2]:
df = pd.read_pickle('D://Datasets//full_data101x56_2_xc_fixed_calcs')
df = df_utils.filter_df(df,ymax=5,max_ar=1.1,radius_std=5)
df = df[(df.cell=='hl60')|(df.cell=='hl60d')]
df = df[np.logical_not((df.cell=='hl60')&(df.date=='11-3-20')&(df.run=='0'))]
df = df[np.logical_not((df.cell=='hl60')&(df.date=='11-5-20')&(df.run=='3'))]
print(len(df))

Length prefilter: 2337
Length postfilter: 1992
1564


In [3]:
df.columns

Index(['event', 'tf', 'mask', 'perimeter', 'area', 'circ', 'deform', 'r_um',
       'xcm_um', 'yc_um', 'perimeter_cx', 'area_cx', 'circ_cx', 'deform_cx',
       'r_um_cx', 'xc_um_el', 'yc_um_el', 'a', 'b', 'aspect', 'r_um_el',
       'cell', 'date', 'run', 'r_idx', 'nar1_idx', 'nar2_idx', 'cav_idx',
       'out1_idx', 'out2_idx', 'rad', 'nar1_def', 'nar2_def', 'cav1_def',
       'r_el', 'nar1_asp', 'nar2_asp', 'cav1_asp', 'nar1_max_arg',
       'nar2_max_arg', 'cav1_min_arg', 'x_poly1', 'x_poly2', 'region1_dx',
       'region1_dt', 'region1_dasp', 'delta_asp', 'v_avg', 'mean_area',
       'mean_perimeter', 'mean_aspect'],
      dtype='object')

In [4]:
df['r1_slope']=df.apply(lambda a: a.x_poly1[0],axis=1)

In [6]:
df.cell.astype('category').cat.codes.to_numpy()

array([0, 0, 0, ..., 1, 1, 1], dtype=int8)

In [9]:
feature_list = ['nar1_asp', 'nar2_asp', 'rad']

# Extract features
x = df[feature_list].to_numpy()
y = df.cell.astype('category').cat.codes.to_numpy()

# Normalize and standardize first
scalar = sklearn.preprocessing.StandardScaler()
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=123)

# Fit scalar on training, apply transformation to val/test
scalar.fit(x_train)
x_train = scalar.transform(x_train)
x_val = scalar.transform(x_val)

x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, random_state=123)

In [11]:
clf = KNeighborsClassifier()
clf.fit(x_train, y_train.ravel())

KNeighborsClassifier()

In [12]:
clf.score(x_val, y_val)

0.6297872340425532

In [19]:
list(itertools.combinations(feature_list, 1))

[('nar1_asp',), ('nar2_asp',), ('rad',)]

In [72]:
feature_list = ['nar1_asp', 'nar2_asp', 'rad','r1_slope','delta_asp']
label = []
score = []
auc = []
aic = []

for i in range(len(feature_list)):
    
    sub_feats = list(itertools.combinations(feature_list, i+1))
    
    for j,feats in enumerate(sub_feats):
        x = df[list(feats)].to_numpy()
        y = df.cell.astype('category').cat.codes.to_numpy()
        
        # Normalize and standardize first
        scalar = sklearn.preprocessing.StandardScaler()
        x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=123)

        # Fit scalar on training, apply transformation to val/test
        scalar.fit(x_train)
        x_train = scalar.transform(x_train)
        x_val = scalar.transform(x_val)
        
        clf = KNeighborsClassifier(n_neighbors=10)
        clf.fit(x_train, y_train.ravel())
        
        score.append(clf.score(x_val, y_val))
        label.append(list(feats))
        
        auc.append(roc_auc_score(y_val, clf.predict_proba(x_val)[:,1]))
        
        #y_hat = logreg.predict(x_val)
        #resid = y_val - y_hat
        #sse = sum(resid**2)
        #k= 2
        #AIC= 2*k - 2*np.log(sse)
        #aic.append(AIC)
        
print(np.max(auc))
print(label[np.argmax(auc)])
print(score[np.argmax(auc)])

0.744272872263303
['nar2_asp', 'rad', 'r1_slope', 'delta_asp']
0.676595744680851


In [73]:
feature_list = ['nar1_asp', 'nar2_asp', 'rad','r1_slope','delta_asp']
label = []
score = []
auc = []
aic = []

for i in range(len(feature_list)):
    
    sub_feats = list(itertools.combinations(feature_list, i+1))
    
    for j,feats in enumerate(sub_feats):
        x = df[list(feats)].to_numpy()
        y = df.cell.astype('category').cat.codes.to_numpy()
        
        # Normalize and standardize first
        scalar = sklearn.preprocessing.StandardScaler()
        x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=123)

        # Fit scalar on training, apply transformation to val/test
        scalar.fit(x_train)
        x_train = scalar.transform(x_train)
        x_val = scalar.transform(x_val)
        
        logreg = LogisticRegression()
        logreg.fit(x_train,y_train)
        
        score.append(logreg.score(x_val, y_val))
        label.append(list(feats))
        
        auc.append(roc_auc_score(y_val, logreg.predict_proba(x_val)[:,1]))
        
        #y_hat = logreg.predict(x_val)
        #resid = y_val - y_hat
        #sse = sum(resid**2)
        #k= 2
        #AIC= 2*k - 2*np.log(sse)
        #aic.append(AIC)

print(np.max(auc))
print(label[np.argmax(auc)])
print(score[np.argmax(auc)])

0.7258046976946498
['nar1_asp', 'rad', 'r1_slope']
0.6723404255319149


In [64]:
label[np.argmax(auc)]

['nar1_asp', 'rad', 'r1_slope']

In [65]:
score[np.argmax(auc)]

0.6723404255319149

In [ ]:
def knn_search(feats)

    # Extract features
    x = df[feature_list].to_numpy()
    y = df[['y']].to_numpy()

    # Normalize and standardize first
    scalar = sklearn.preprocessing.StandardScaler()
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=123)

    # Fit scalar on training, apply transformation to val/test
    scalar.fit(x_train)
    x_train = scalar.transform(x_train)
    x_val = scalar.transform(x_val)

    x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, random_state=123)

    # Grid search all hyperparameters
    leaf_size = list(range(1, 200, 5))
    neighbors = np.arange(1, 25)
    score = {}
    for leaf in leaf_size:
        for n in neighbors:
            clf = KNeighborsClassifier(n_neighbors=n, leaf_size=leaf)
            clf.fit(x_train, y_train.ravel())
            score[(str(n), str(leaf))] = clf.score(x_val, y_val)

    # Select best weights
    n, leaf = max(score, key=lambda key: score[key])

    print('Optimized hyper params:')
    print('N: ', n, '\n', 'Leaf size: ', leaf)
    clf = KNeighborsClassifier(n_neighbors=int(n), leaf_size=int(leaf))
    clf.fit(x_train, y_train.ravel())
    print(clf.score(x_test, y_test))

    sklearn.metrics.plot_roc_curve(clf, x_test, y_test)
    plt.title('k Nearest Neighbors' + ' K: ' + str(n) + ' Leaf Size: ' + str(leaf))
    plt.savefig('knn_roc.png', dpi=300)

    pickle.dump(clf, open('knn.pkl', 'wb'))